In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# MeCabとtransformersを用意する
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# 以下で報告があるようにmecab-python3のバージョンを0.996.5にしないとtokezerで落ちる
# https://stackoverflow.com/questions/62860717/huggingface-for-japanese-tokenizer
!pip install mecab-python3==0.996.5
!pip install unidic-lite # これないとMeCab実行時にエラーで落ちる
!pip install transformers

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl lib

In [57]:
import pickle
import pandas as pd

# データセット格納先
drive_dir = "drive/My Drive/Colab Notebooks/livedoor_data/"

with open(drive_dir + "livedoor_title_category.pickle", 'rb') as f:
  livedoor_data = pickle.load(f)

# livedoor_data.head()

# カテゴリーのリストをデータセットから取得
categories = list(set(livedoor_data['category']))
print(categories)

# カテゴリーのID辞書を作成
id2cat = dict(zip(list(range(len(categories))), categories))
cat2id = dict(zip(categories, list(range(len(categories)))))
print(id2cat)
print(cat2id)

# DataFrameにカテゴリーID列を追加
livedoor_data['category_id'] = livedoor_data['category'].map(cat2id)

# 念の為シャッフル
livedoor_data = livedoor_data.sample(frac=1).reset_index(drop=True)

# データセットを本文とカテゴリーID列だけにする
livedoor_data = livedoor_data[['title', 'category_id']]
# # 本文からURLやタイトルを取り除く
# livedoor_data['body'] = livedoor_data['body'].map(lambda x: "".join(x.split("\n")[3:]))
livedoor_data.head()

['kaden-channel', 'dokujo-tsushin', 'peachy', 'movie-enter', 'smax', 'livedoor-homme', 'it-life-hack', 'topic-news', 'sports-watch']
{0: 'kaden-channel', 1: 'dokujo-tsushin', 2: 'peachy', 3: 'movie-enter', 4: 'smax', 5: 'livedoor-homme', 6: 'it-life-hack', 7: 'topic-news', 8: 'sports-watch'}
{'kaden-channel': 0, 'dokujo-tsushin': 1, 'peachy': 2, 'movie-enter': 3, 'smax': 4, 'livedoor-homme': 5, 'it-life-hack': 6, 'topic-news': 7, 'sports-watch': 8}


,title,category_id
0,Twitterでまた事件！ 今度はビビる大木が悪口を言うショップ店員に怒り心頭！【話題】,0
1,テレビ朝日で花見のニュースに津波の映像が流れる放送事故 ネット掲示板は一時騒然,7
2,【Sports Watch】シャワポワを追いつめた17歳美女とは？,8
3,千葉県船橋市で毎時5.82マイクロシーベルトを検出,7
4,人恋しい季節の到来−冬にひとりはさみしいヨネ！,1


In [58]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(livedoor_data, train_size=0.8)
print("学習データサイズ", train_df.shape[0])
print("テストデータサイズ", test_df.shape[0])

# tsvファイルとして保存する
train_df.to_csv(drive_dir + 'train.tsv', sep='\t', index=False, header=None)
test_df.to_csv(drive_dir + 'test.tsv', sep='\t', index=False, header=None)

学習データサイズ 5900
テストデータサイズ 1476


In [59]:
# Download from model library from huggingface.co
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
import torchtext

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


# 公式では以下のようにAutoTokenizerでtokenizerを宣言しますが上記のtokenizerと同様なので、上記のtokenizerを使い回す。
# distil_tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

def bert_tokenizer(text):
  return tokenizer.encode(text, return_tensors='pt')[0]

In [60]:
# torchtextを使って、学習データとテストデータのイテレータを作成

TEXT = torchtext.data.Field(sequential=True, tokenize=bert_tokenizer, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, pad_token=0)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

train_data, test_data = torchtext.data.TabularDataset.splits(
    path=drive_dir, train='train.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

BATCH_SIZE = 32
train_iter, test_iter = torchtext.data.Iterator.splits((train_data, test_data), batch_sizes=(BATCH_SIZE, BATCH_SIZE), repeat=False, sort=False)

In [63]:
import torch
from torch import nn
import torch.nn.functional as F

# DistilBERT
# https://github.com/BandaiNamcoResearchInc/DistilBERT-base-jp

class DistilBertClassifier(nn.Module):
  def __init__(self):
    super(DistilBertClassifier, self).__init__()
    self.distil_bert = AutoModel.from_pretrained("bandainamco-mirai/distilbert-base-japanese")
    # BERTの隠れ層の次元数は768, livedoorニュースのカテゴリ数が9
    self.linear = nn.Linear(768, 9)
    # 重み初期化処理
    nn.init.normal_(self.linear.weight, std=0.02)
    nn.init.normal_(self.linear.bias, 0)

  def forward(self, input_ids):
    # last_hidden_stateとattentionsを受け取る
    vec, _ = self.distil_bert(input_ids)
    # 先頭トークンclsのベクトルだけ取得
    vec = vec[:,0,:]
    vec = vec.view(-1, 768)
    # 全結合層でクラス分類用に次元を変換
    out = self.linear(vec)
    return F.log_softmax(out)

distil_classifier = DistilBertClassifier()
# distil_classifier.distil_bert

In [65]:
# ファインチューニングの設定
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# まずは全部OFF
for param in distil_classifier.parameters():
    param.requires_grad = False

# BERTの最後の層だけ更新ON
for param in distil_classifier.distil_bert.transformer.layer[-1].parameters():
    param.requires_grad = True

# クラス分類のところもON
for param in distil_classifier.linear.parameters():
    param.requires_grad = True

import torch.optim as optim

# 事前学習済の箇所は学習率小さめ、最後の全結合層は大きめにする。
optimizer = optim.Adam([
    {'params': distil_classifier.distil_bert.transformer.layer[-1].parameters(), 'lr': 5e-5},
    {'params': distil_classifier.linear.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
loss_function = nn.NLLLoss()

In [66]:
import time

# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# ネットワークをGPUへ送る
distil_classifier.to(device)
losses = []

start = time.time()
# エポック数は10で
for epoch in range(10):
  all_loss = 0
  for idx, batch in enumerate(train_iter):
    batch_loss = 0
    distil_classifier.zero_grad()
    input_ids = batch.Text[0].to(device)
    label_ids = batch.Label.to(device)
    out = distil_classifier(input_ids)
    batch_loss = loss_function(out, label_ids)
    batch_loss.backward()
    optimizer.step()
    all_loss += batch_loss.item()
  print("epoch", epoch, "\t" , "loss", all_loss)

end = time.time()
print ("time : ", end - start)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 450.1027842760086
epoch 1 	 loss 317.39041769504547
epoch 2 	 loss 211.34138756990433
epoch 3 	 loss 144.4813650548458
epoch 4 	 loss 106.24609130620956
epoch 5 	 loss 83.87273170053959
epoch 6 	 loss 68.9661111086607
epoch 7 	 loss 59.31868125498295
epoch 8 	 loss 49.874382212758064
epoch 9 	 loss 41.56027300283313
time :  60.22182369232178


In [67]:

from sklearn.metrics import classification_report

answer = []
prediction = []
with torch.no_grad():
    for batch in test_iter:

        text_tensor = batch.Text[0].to(device)
        label_tensor = batch.Label.to(device)

        score = distil_classifier(text_tensor)
        _, pred = torch.max(score, 1)

        prediction += list(pred.cpu().numpy())
        answer += list(label_tensor.cpu().numpy())
print(classification_report(prediction, answer, target_names=categories))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


                precision    recall  f1-score   support

 kaden-channel       0.93      0.96      0.95       163
dokujo-tsushin       0.88      0.88      0.88       178
        peachy       0.86      0.75      0.80       202
   movie-enter       0.86      0.84      0.85       183
          smax       0.96      0.95      0.95       165
livedoor-homme       0.67      0.71      0.69        96
  it-life-hack       0.91      0.91      0.91       178
    topic-news       0.80      0.86      0.83       148
  sports-watch       0.88      0.91      0.89       163

      accuracy                           0.87      1476
     macro avg       0.86      0.86      0.86      1476
  weighted avg       0.87      0.87      0.87      1476



In [68]:
class BertClassifier(nn.Module):
  def __init__(self):
    super(BertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
    # BERTの隠れ層の次元数は768, livedoorニュースのカテゴリ数が9
    self.linear = nn.Linear(768, 9)
    # 重み初期化処理
    nn.init.normal_(self.linear.weight, std=0.02)
    nn.init.normal_(self.linear.bias, 0)

  def forward(self, input_ids):
    # last_hidden_stateとattentionsを受け取る
    vec, _ = self.bert(input_ids)
    # 先頭トークンclsのベクトルだけ取得
    vec = vec[:,0,:]
    vec = vec.view(-1, 768)
    # 全結合層でクラス分類用に次元を変換
    out = self.linear(vec)
    return F.log_softmax(out)

bert_classifier = BertClassifier()

# ファインチューニングの設定
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# まずは全部OFF
for param in bert_classifier.parameters():
    param.requires_grad = False

# BERTの最後の層だけ更新ON
for param in bert_classifier.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# クラス分類のところもON
for param in bert_classifier.linear.parameters():
    param.requires_grad = True

import torch.optim as optim

# 事前学習済の箇所は学習率小さめ、最後の全結合層は大きめにする。
optimizer = optim.Adam([
    {'params': bert_classifier.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': bert_classifier.linear.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
loss_function = nn.NLLLoss()

In [69]:
import time

start = time.time()
# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# ネットワークをGPUへ送る
bert_classifier.to(device)
losses = []

# エポック数は5で
for epoch in range(10):
  all_loss = 0
  for idx, batch in enumerate(train_iter):
    batch_loss = 0
    bert_classifier.zero_grad()
    input_ids = batch.Text[0].to(device)
    label_ids = batch.Label.to(device)
    out = bert_classifier(input_ids)
    batch_loss = loss_function(out, label_ids)
    batch_loss.backward()
    optimizer.step()
    all_loss += batch_loss.item()
  print("epoch", epoch, "\t" , "loss", all_loss)

end = time.time()

print ("time : ", end - start)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 251.19750046730042
epoch 1 	 loss 110.7038831859827
epoch 2 	 loss 82.88570280373096
epoch 3 	 loss 67.0771074667573
epoch 4 	 loss 56.24497305601835
epoch 5 	 loss 42.61423560976982
epoch 6 	 loss 35.98485875874758
epoch 7 	 loss 25.728398952633142
epoch 8 	 loss 20.40780107676983
epoch 9 	 loss 16.567239843308926
time :  101.97362518310547


In [70]:
answer = []
prediction = []
with torch.no_grad():
    for batch in test_iter:

        text_tensor = batch.Text[0].to(device)
        label_tensor = batch.Label.to(device)

        score = bert_classifier(text_tensor)
        _, pred = torch.max(score, 1)

        prediction += list(pred.cpu().numpy())
        answer += list(label_tensor.cpu().numpy())
print(classification_report(prediction, answer, target_names=categories))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


                precision    recall  f1-score   support

 kaden-channel       0.94      0.92      0.93       172
dokujo-tsushin       0.75      0.86      0.80       156
        peachy       0.81      0.68      0.74       211
   movie-enter       0.78      0.81      0.80       171
          smax       0.98      0.91      0.94       176
livedoor-homme       0.68      0.83      0.75        83
  it-life-hack       0.79      0.94      0.86       150
    topic-news       0.81      0.76      0.78       172
  sports-watch       0.89      0.82      0.85       185

      accuracy                           0.83      1476
     macro avg       0.83      0.84      0.83      1476
  weighted avg       0.84      0.83      0.83      1476



In [71]:
# https://www.ai-shift.co.jp/techblog/635

class BertClassifierRevised(nn.Module):
  def __init__(self):
    super(BertClassifierRevised, self).__init__()
    self.bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
    # BERTの隠れ層の次元数は768, livedoorニュースのカテゴリ数が9
    self.linear = nn.Linear(768*4, 9)
    # 重み初期化処理
    nn.init.normal_(self.linear.weight, std=0.02)
    nn.init.normal_(self.linear.bias, 0)
  
  def _get_cls_vec(self, vec):
    return vec[:,0,:].view(-1, 768)

  def forward(self, input_ids):
    # last_hidden_stateとattentionsを受け取る
    _, _,  hidden_states = self.bert(input_ids, output_hidden_states=True)
    # 先頭トークンclsのベクトルだけ取得
    vec1 = self._get_cls_vec(hidden_states[-1])
    vec2 = self._get_cls_vec(hidden_states[-2])
    vec3 = self._get_cls_vec(hidden_states[-3])
    vec4 = self._get_cls_vec(hidden_states[-4])
    vec = torch.cat([vec1, vec2, vec3, vec4], dim=1)
    # 全結合層でクラス分類用に次元を変換
    out = self.linear(vec)
    return F.log_softmax(out)

bert_classifier_revised = BertClassifierRevised()

# ファインチューニングの設定
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# まずは全部OFF
for param in bert_classifier_revised.parameters():
    param.requires_grad = False

# BERTの最後の層だけ更新ON
for param in bert_classifier_revised.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

for param in bert_classifier_revised.bert.encoder.layer[-2].parameters():
    param.requires_grad = True

for param in bert_classifier_revised.bert.encoder.layer[-3].parameters():
    param.requires_grad = True

for param in bert_classifier_revised.bert.encoder.layer[-4].parameters():
    param.requires_grad = True

# クラス分類のところもON
for param in bert_classifier_revised.linear.parameters():
    param.requires_grad = True

import torch.optim as optim

# 事前学習済の箇所は学習率小さめ、最後の全結合層は大きめにする。
optimizer = optim.Adam([
    {'params': bert_classifier_revised.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': bert_classifier_revised.bert.encoder.layer[-2].parameters(), 'lr': 5e-5},
    {'params': bert_classifier_revised.bert.encoder.layer[-3].parameters(), 'lr': 5e-5},
    {'params': bert_classifier_revised.bert.encoder.layer[-4].parameters(), 'lr': 5e-5},
    {'params': bert_classifier_revised.linear.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
loss_function = nn.NLLLoss()

In [72]:
import time

start = time.time()
# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# ネットワークをGPUへ送る
bert_classifier_revised.to(device)
losses = []

# エポック数は5で
for epoch in range(10):
  all_loss = 0
  for idx, batch in enumerate(train_iter):
    batch_loss = 0
    bert_classifier_revised.zero_grad()
    input_ids = batch.Text[0].to(device)
    label_ids = batch.Label.to(device)
    out = bert_classifier_revised(input_ids)
    batch_loss = loss_function(out, label_ids)
    batch_loss.backward()
    optimizer.step()
    all_loss += batch_loss.item()
  print("epoch", epoch, "\t" , "loss", all_loss)

end = time.time()

print ("time : ", end - start)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 196.0047192275524
epoch 1 	 loss 75.8067753687501
epoch 2 	 loss 42.30751228891313
epoch 3 	 loss 16.470114511903375
epoch 4 	 loss 7.427484432584606
epoch 5 	 loss 2.9392087209271267
epoch 6 	 loss 1.5984382012393326
epoch 7 	 loss 1.7370687873335555
epoch 8 	 loss 0.9278695838729618
epoch 9 	 loss 1.499190401067608
time :  149.01919651031494


In [73]:
answer = []
prediction = []
with torch.no_grad():
    for batch in test_iter:

        text_tensor = batch.Text[0].to(device)
        label_tensor = batch.Label.to(device)

        score = bert_classifier_revised(text_tensor)
        _, pred = torch.max(score, 1)

        prediction += list(pred.cpu().numpy())
        answer += list(label_tensor.cpu().numpy())
print(classification_report(prediction, answer, target_names=categories))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


                precision    recall  f1-score   support

 kaden-channel       0.80      0.99      0.89       137
dokujo-tsushin       0.89      0.86      0.88       183
        peachy       0.78      0.82      0.80       168
   movie-enter       0.87      0.88      0.87       176
          smax       0.95      0.93      0.94       168
livedoor-homme       0.72      0.83      0.77        88
  it-life-hack       0.95      0.79      0.86       215
    topic-news       0.83      0.84      0.83       159
  sports-watch       0.92      0.86      0.89       182

      accuracy                           0.86      1476
     macro avg       0.86      0.87      0.86      1476
  weighted avg       0.87      0.86      0.86      1476



In [42]:
class BertAndDistilBertClassifier(nn.Module):
  def __init__(self):
    super(BertAndDistilBertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
    self.distil_bert = AutoModel.from_pretrained("bandainamco-mirai/distilbert-base-japanese")
    # BERTの隠れ層の次元数は768, livedoorニュースのカテゴリ数が9
    self.linear = nn.Linear(768*2, 9)
    # 重み初期化処理
    nn.init.normal_(self.linear.weight, std=0.02)
    nn.init.normal_(self.linear.bias, 0)
  
  def _get_cls_vec(self, vec):
    return vec[:,0,:].view(-1, 768)

  def forward(self, input_ids):
    # last_hidden_stateとattentionsを受け取る
    bert_vec, _ = self.bert(input_ids)
    distil_vec, _ = self.distil_bert(input_ids)
    bert_cls_vec = self._get_cls_vec(bert_vec)
    distil_cls_vec = self._get_cls_vec(distil_vec)
    vec = torch.cat([bert_cls_vec, distil_cls_vec], dim=1)
    out = self.linear(vec)
    return F.log_softmax(out)


bert_and_distil_classifier = BertAndDistilBertClassifier()

# ファインチューニングの設定
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# まずは全部OFF
for param in bert_and_distil_classifier.parameters():
    param.requires_grad = False

# BERTの最後の層だけ更新ON
for param in bert_and_distil_classifier.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# DistilBETの最後の層だけ更新ON
for param in bert_and_distil_classifier.distil_bert.transformer.layer[-1].parameters():
    param.requires_grad = True

# クラス分類のところもON
for param in bert_and_distil_classifier.linear.parameters():
    param.requires_grad = True

import torch.optim as optim

# 事前学習済の箇所は学習率小さめ、最後の全結合層は大きめにする。
optimizer = optim.Adam([
    {'params': bert_and_distil_classifier.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': bert_and_distil_classifier.distil_bert.transformer.layer[-1].parameters(), 'lr': 5e-5},
    {'params': bert_and_distil_classifier.linear.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
loss_function = nn.NLLLoss()

In [43]:
import time

start = time.time()
# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# ネットワークをGPUへ送る
bert_and_distil_classifier.to(device)
losses = []

# エポック数は5で
for epoch in range(10):
  all_loss = 0
  for idx, batch in enumerate(train_iter):
    batch_loss = 0
    bert_and_distil_classifier.zero_grad()
    input_ids = batch.Text[0].to(device)
    label_ids = batch.Label.to(device)
    out = bert_and_distil_classifier(input_ids)
    batch_loss = loss_function(out, label_ids)
    batch_loss.backward()
    optimizer.step()
    all_loss += batch_loss.item()
  print("epoch", epoch, "\t" , "loss", all_loss)

end = time.time()

print ("time : ", end - start)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 	 loss 253.8527083992958
epoch 1 	 loss 108.67510583996773
epoch 2 	 loss 80.17743158340454
epoch 3 	 loss 61.985263999551535
epoch 4 	 loss 49.55005703121424
epoch 5 	 loss 37.398751348257065
epoch 6 	 loss 29.679549926891923
epoch 7 	 loss 20.620460195466876
epoch 8 	 loss 14.49152850266546
epoch 9 	 loss 11.417884467169642
time :  157.13771057128906


In [44]:
answer = []
prediction = []
with torch.no_grad():
    for batch in test_iter:

        text_tensor = batch.Text[0].to(device)
        label_tensor = batch.Label.to(device)

        score = bert_and_distil_classifier(text_tensor)
        _, pred = torch.max(score, 1)

        prediction += list(pred.cpu().numpy())
        answer += list(label_tensor.cpu().numpy())
print(classification_report(prediction, answer, target_names=categories))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


                precision    recall  f1-score   support

 kaden-channel       0.95      0.90      0.92       171
dokujo-tsushin       0.84      0.80      0.82       191
        peachy       0.78      0.69      0.73       178
   movie-enter       0.77      0.85      0.81       162
          smax       0.95      0.96      0.96       185
livedoor-homme       0.80      0.77      0.78       118
  it-life-hack       0.79      0.96      0.86       136
    topic-news       0.88      0.81      0.84       173
  sports-watch       0.87      0.94      0.90       162

      accuracy                           0.85      1476
     macro avg       0.85      0.85      0.85      1476
  weighted avg       0.85      0.85      0.85      1476



In [45]:
tokenizer = AutoTokenizer.from_pretrained("ALINEAR/albert-japanese")


Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


In [19]:
# import lightgbm as lgbm

# preds = []
# answer = []
# for title, cat_id in zip(ensemble_df['title'], ensemble_df['category_id']):
#   with torch.no_grad():
#     input_ids = tokenizer.encode(title, return_tensors='pt').to(device)
#     bert_score = bert_classifier(input_ids)
#     _, bert_pred = torch.max(bert_score, 1)
#     distil_score = distil_classifier(input_ids)
#     _, distil_pred = torch.max(distil_score, 1)
#     preds.append([bert_pred.item(), distil_pred.item()])
#     answer.append(cat_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [20]:
# import pandas as pd
# X = pd.DataFrame(preds, columns=['bert', 'distil'])
# lgbm_model = lgbm.LGBMClassifier()
# lgbm_model.fit(X, answer)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [21]:
# answer = []
# bert_preds = []
# distil_preds = []
# with torch.no_grad():
#     for batch in test_iter:

#         text_tensor = batch.Text[0].to(device)
#         label_tensor = batch.Label.to(device)

#         bert_score = bert_classifier(text_tensor)
#         _, bert_pred = torch.max(bert_score, 1)

#         distil_score = distil_classifier(text_tensor)
#         _, distil_pred = torch.max(distil_score, 1)

#         bert_preds += list(bert_pred.cpu().numpy())
#         distil_preds += list(distil_pred.cpu().numpy())

#         answer += list(label_tensor.cpu().numpy())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [22]:
# X = pd.DataFrame()
# X['bert'] = bert_preds
# X['distil'] = distil_preds
# lgbm_pred = lgbm_model.predict(X)
# print(classification_report(lgbm_pred, answer, target_names=categories))

                precision    recall  f1-score   support

 kaden-channel       0.88      0.97      0.92       170
dokujo-tsushin       0.91      0.69      0.79       232
        peachy       0.60      0.68      0.64       155
   movie-enter       0.83      0.86      0.84       155
          smax       0.95      0.97      0.96       173
livedoor-homme       0.54      0.93      0.68        57
  it-life-hack       0.91      0.86      0.88       200
    topic-news       0.84      0.79      0.81       156
  sports-watch       0.91      0.87      0.89       178

      accuracy                           0.84      1476
     macro avg       0.82      0.85      0.82      1476
  weighted avg       0.85      0.84      0.84      1476

